In [1]:
import numpy as np
import time
import os
import h5py
import inspect
from tqdm import tqdm
import sys

import presto
from presto import lockin, utils
from presto.hardware import AdcFSample, AdcMode, DacFSample, DacMode


In [2]:
# # DSP e chunk
# nr_sig_freqs = 81
# fNCO         = 4.1e9 
# fNCO_pump    = 8.3e9
# df           = 100e3
# Npix         = 4_000_000
# N_chunk      = 1_000
# Navg         = 1

# # Parametric pump
# _fp_center   = 8.4e9
# nr_sq_freqs  = 4
# pump_max     = 0.1
# pump_min     = 0.0
# pump_steps   = 11
# amp_pump_arr = np.linspace(pump_min, pump_max, pump_steps) # np.array([0.05]) 
# k_array      = np.array([-9, -1, 1, 9])
# output_amp   = 0.0 

# # Frequency Comb
# fs_center = _fp_center / 2 - fNCO
# fs_comb = fs_center + df * np.linspace( int(-(nr_sig_freqs)/2),
#                                         int((nr_sig_freqs)/2), 
#                                         nr_sig_freqs) +fNCO

# # Squeezing Pumps
# f_center  = fs_center + fNCO
# fp_center = 2 * f_center - fNCO_pump
# fp_comb   = fp_center + 1 * df * k_array + fNCO_pump

# # Set pumps phases
# phases_arr             = np.array([np.pi, 0, 0, 0])
# phases_q_arr           = phases_arr - np.pi/2

# print(fs_comb)
# print(fp_comb)
# print(phases_arr)


In [ ]:
def save_script(folder, file, sample_, myrun_, myrun_attrs_):
    if not os.path.isdir(folder):
        os.makedirs(folder)
    with h5py.File(os.path.join(folder, file), "a") as savefile:
        run_str = f"{sample_}/{myrun_}"
        # Assicurati che il gruppo esista
        if run_str not in savefile:
            savefile.create_group(run_str)
        for key, val in myrun_attrs_.items():
            savefile[run_str].attrs[key] = val
    print("Saved run attributes.")

# Save data function
def save_data(folder, file, sample_, myrun_, freq, usb_on_arr, usb_off_arr, index_dict):
    if not os.path.isdir(folder):
        os.makedirs(folder)

    # Open the save file (.hdf5) in append mode
    with h5py.File(os.path.join(folder, file), "a") as savefile:
        # Determine index number under run
        idx_amp = index_dict["idx_amp"]

        # String as handles
        freq_data_str = "{}/{}/{}/freq comb".format(sample_, myrun_, str(idx_amp))
        usb_on_data_str = "{}/{}/{}/USB ON".format(sample_, myrun_, str(idx_amp))
        usb_off_data_str = "{}/{}/{}/USB OFF".format(sample_, myrun_, str(idx_amp))
        idx_amp_attrs_str = "{}/{}/{}/".format(sample_, myrun_, str(idx_amp))

        # Write data to datasets
        savefile.create_dataset(freq_data_str, (np.shape(freq)),
                                dtype=float, data=freq)
        savefile.create_dataset(usb_on_data_str, (np.shape(usb_on_arr)),
                                dtype=complex, data=usb_on_arr)
        savefile.create_dataset(usb_off_data_str, (np.shape(usb_off_arr)),
                                dtype=complex, data=usb_off_arr)

        # Write index attributes
        savefile[idx_amp_attrs_str].attrs["pump amp"] = index_dict["pump_amp"]

        # Write dataset attributes
        savefile[freq_data_str].attrs["Unit"] = "Hz"
        savefile[usb_on_data_str].attrs["Unit"] = "fsu complex"
        savefile[usb_off_data_str].attrs["Unit"] = "fsu complex"


save_folder = r'I:/JPA-Data/2025-09'
Ym_str = time.strftime("%Y-%m")
save_file   = r"{}.hdf5".format(Ym_str)  
myrun       = time.strftime("%Y-%m-%d_%H_%M_%S")

sample      = 'JPA'
meas_type   = 'Entanglement'
atten       = 80
temperature = 0.0107

ADDRESS = '130.237.35.90'
PORT    = 42873
Box     = 'Presto DELTA'

input_port   = 1
output_port  = 1
flux_port    = 5
bias_port    = 4

DAC_CURRENT  = 32_000  # µA
bias_val     = 1.5275   # FS     

#Square Lattice dimensions (Nx must be odd)
Ntot = 49
Nxy = 7 #int(np.sqrt(Ntot))  # Number of modes/sites in x direction
ky_pmp=Nxy

# DSP e chunk
nr_sig_freqs = Ntot
fNCO         = 4.1e9 
fNCO_pump    = 8.3e9
df           = 100e3
Npix         = 4_000_000
N_chunk      = 1_000
Navg         = 1


# Parametric pump
_fp_center   = 8.4e9
nr_sq_freqs  = 4
pump_max     = 0.1
pump_min     = 0.0
pump_steps   = 11
amp_pump_arr = np.linspace(pump_min, pump_max, pump_steps) # np.array([0.05]) # 0.04*np.ones(40) 
k_array      = np.array([-ky_pmp, -1, 1, ky_pmp])
output_amp   = 0.0 

# Array for storing measurement data
usb_ON_arr  = np.zeros((int(Npix//2),
                        nr_sig_freqs), 
                        dtype=complex)
lsb_ON_arr = np.zeros_like(usb_ON_arr)
usb_OFF_arr = np.zeros_like(usb_ON_arr)
lsb_OFF_arr = np.zeros_like(usb_ON_arr)

# Propagation Phase Drift
input_phases = np.zeros(nr_sig_freqs)
dPhiDrift_df = 1.8869636024e-6  # rad/Hz - Measured Drift in phase per Hz 
# dPhiDrift_df = 1.8964498850e-6  # rad/Hz - Measured Drift in phase per Hz for df=1MHz

# Lock-in amplifier initialization
with lockin.Lockin(address=ADDRESS,
                   port=PORT,
                   adc_mode=AdcMode.Mixed,
                   adc_fsample=AdcFSample.G2,
                   dac_mode=[DacMode.Mixed02, DacMode.Mixed04, DacMode.Mixed02, DacMode.Mixed02],
                   dac_fsample=[DacFSample.G6, DacFSample.G10, DacFSample.G6, DacFSample.G6],
                   ) as lck:
    
    # Start timer
    t_start = time.strftime("%Y-%m-%d_%H_%M_%S")

    lck.hardware.set_dac_current(output_port, DAC_CURRENT)
    lck.hardware.set_dc_bias(bias_val, bias_port)
    lck.hardware.sleep(1.0, False)

    lck.hardware.configure_mixer(freq=fNCO_pump,
                                 out_ports=flux_port,
                                 sync=False)

    lck.hardware.configure_mixer(freq=fNCO,
                                 in_ports=input_port,
                                 out_ports=output_port,
                                 sync=True)

    # Frequency Comb
    _fs_center = _fp_center / 2 - fNCO
    fs_center, df = lck.tune(_fs_center, df)
    fs_comb = fs_center + df * np.linspace( int(-(nr_sig_freqs)/2),
                                            int((nr_sig_freqs)/2), 
                                            nr_sig_freqs)

    # Squeezing Pumps
    f_center  = fs_center + fNCO
    fp_center = 2 * f_center - fNCO_pump
    fp_comb   = fp_center + 1 * df * k_array

    # Set pumps phases
    phases_arr             = np.array([np.pi, 0, 0, 0])
    phases_q_arr           = phases_arr - np.pi/2

    lck.set_df(df)

    # Output group for Squeezing Pumps
    og_pump = lck.add_output_group(ports=flux_port, nr_freq=nr_sq_freqs)
    og_pump.set_frequencies(fp_comb)
    og_pump.set_phases(phases=phases_arr, phases_q=phases_q_arr)

    # Input group
    ig = lck.add_input_group(port=input_port, nr_freq=nr_sig_freqs)
    ig.set_frequencies(fs_comb)
    # Set input phases
    input_phases[0]=0.0
    for i in range(1, nr_sig_freqs):
        input_phases[i] = input_phases[i-1] - dPhiDrift_df * df  # Compensate the measured phase delay drift between modes.
    ig.set_phases(phases=input_phases.copy()) 

    lck.apply_settings()
    time.sleep(0.1)

    total_iters = len(amp_pump_arr) * (Npix // N_chunk)
    with tqdm(total=total_iters, ncols=80) as pbar:
        
        for amp_idx, amp_val in enumerate(amp_pump_arr):

            n_on = n_off = 0
            
            for n in range(0, Npix, N_chunk):
                
                if (n // N_chunk) % 2 == 0:
                    
                    n_on +=N_chunk
                    og_pump.set_amplitudes(amp_val * np.ones(nr_sq_freqs))
                    
                else:
                    
                    n_off += N_chunk
                    og_pump.set_amplitudes(np.zeros(nr_sq_freqs))

                lck.apply_settings()
                time.sleep(0.01)

                # Get lock-in packets (pixels) from the local buffer
                data = lck.get_pixels(N_chunk, summed=False, nsum=Navg)
                freqs, pixels_i, pixels_q = data[input_port]

                # Convert a measured IQ pair into a low/high sideband pair
                LSB, HSB = utils.untwist_downconversion(pixels_i, pixels_q)

                # Store data in array
                if (n // N_chunk) % 2 == 0:
                    usb_ON_arr[n_on - N_chunk:n_on] = HSB
                    lsb_ON_arr[n_on - N_chunk:n_on] = LSB
                else:
                    usb_OFF_arr[n_off - N_chunk:n_off] = HSB
                    lsb_OFF_arr[n_off - N_chunk:n_off] = LSB

                # Update progress bar
                pbar.update(1)

            idx_dict = {
                "pump_amp":   amp_val,
                "idx_amp":    amp_idx,
            }
            save_data(save_folder, save_file, meas_type, myrun, fs_comb + fNCO, usb_ON_arr, usb_OFF_arr, idx_dict)


    og_pump.set_amplitudes(np.zeros(nr_sq_freqs))
    lck.apply_settings()
    lck.hardware.set_dc_bias(0.0, bias_port)

# Stop timer
t_end = time.strftime("%Y-%m-%d_%H_%M_%S")

myrun_attrs = {
    "Meas":        meas_type,
    "Instr":       Box,
    "T":           temperature,
    "Sample":      sample,
    "att":         atten,
    "fs_start":    fs_comb[0] + fNCO,
    "fs_stop":     fs_comb[-1] + fNCO,
    "df":          df,
    "nr_sig_freqs":nr_sig_freqs,
    "fp_comb":     fp_comb + fNCO_pump,
    "amp_pump":    amp_pump_arr,
    "phase_pump":  phases_arr,
    "DC bias":     bias_val,
    "Npixels":     Npix,
    "Nchunk":      N_chunk,
    "Naverages":   Navg,
    "t_start":     t_start,
    "t_end":       t_end,
}

save_script(save_folder, save_file, meas_type, myrun, myrun_attrs)



100%|█████████████████████████████████████| 44000/44000 [54:08<00:00, 13.54it/s]

Saved run attributes.
